<a href="https://colab.research.google.com/github/cyborgx0x/deepfake_detection/blob/main/ResNext50_and_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import nn

In [2]:
from torchvision import transforms, models

In [4]:
from torch.utils.data.dataset import Dataset

In [5]:
import torch

In [10]:
import cv2

In [ ]:
!pip install -q gdown

In [8]:
!gdown https://drive.google.com/drive/folders/1UX8jXUXyEjhLLZ38tcgOwGsZ6XFSLDJ- -O /content/model --folder

Retrieving folder list
Processing file 1Klw2YvgxCMoODEwlY0mpaApJ4Q_mVL6c model_84_acc_10_frames_final_data.pt
Processing file 1HqH15cM_Aye4lWLnO4JjMAapgGmOA6Fl model_87_acc_20_frames_final_data.pt
Processing file 1kdBOS9eZ1GSRa4kNPkjhQMWKDkBeAdK3 model_89_acc_40_frames_final_data.pt
Processing file 1-HU636BQ3g6JOCab9dwcQ3QfHTmHBNtq model_90_acc_20_frames_FF_data.pt
Processing file 1nrNmvKTBPc00PvrJDOTDV1Tyh1jtL0Mt model_90_acc_60_frames_final_data.pt
Processing file 1PSvjfeNtxwWi3RjK0AnEB6QGMhBiwsYu model_93_acc_100_frames_celeb_FF_data.pt
Processing file 1n7XEHfyOh85SGL297KCLtXu0wmLwV9gQ model_95_acc_40_frames_FF_data.pt
Processing file 1GQiNcveraGzfhe2kRqgXpwrF8CRdiww5 model_97_acc_60_frames_FF_data.pt
Processing file 1SCcLxNSHA1GZHBCDhoFYgWhenlL8_BdU model_97_acc_80_frames_FF_data.pt
Processing file 1R4LofQFf93V7pZZ32zAHzDCd07JjdnjZ model_97_acc_100_frames_FF_data.pt
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
F

In [3]:
class Model(nn.Module):

    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(x_lstm[:,-1,:]))

In [9]:
model = Model(2).cuda()
ACCURACY = 00


path_to_model = "/content/model/model_87_acc_20_frames_final_data.pt"
model.load_state_dict(torch.load(path_to_model))
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [13]:
preprocessed_images = []
faces_cropped_images = []

In [36]:
cap = cv2.VideoCapture("/content/john.mp4")
frames = []
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frames.append(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()

In [17]:
from PIL import Image as pImage

In [38]:
sequence_length=20
for i in range(1, sequence_length+1):
    frame = frames[i]
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = pImage.fromarray(image, 'RGB')
    image_name = "john_preprocessed_"+str(i)+'.png'
    image_path = "/content/images/" + image_name
    img.save(image_path)
    preprocessed_images.append(image_name)

In [39]:
preprocessed_images

['john_preprocessed_1.png',
 'john_preprocessed_2.png',
 'john_preprocessed_3.png',
 'john_preprocessed_4.png',
 'john_preprocessed_5.png',
 'john_preprocessed_6.png',
 'john_preprocessed_7.png',
 'john_preprocessed_8.png',
 'john_preprocessed_9.png',
 'john_preprocessed_10.png',
 'john_preprocessed_11.png',
 'john_preprocessed_12.png',
 'john_preprocessed_13.png',
 'john_preprocessed_14.png',
 'john_preprocessed_15.png',
 'john_preprocessed_16.png',
 'john_preprocessed_17.png',
 'john_preprocessed_18.png',
 'john_preprocessed_19.png',
 'john_preprocessed_20.png']

In [11]:
!pip install face-recognition
import face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566171 sha256=315f3311a5d439efc4561f2ef0357788e5645bf68efd00b9f24a2c9b1fd5a5fd
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [42]:

padding = 40
faces_found = 0
for i in range(1, sequence_length+1):
    frame = frames[i]
    #fig, ax = plt.subplots(1,1, figsize=(5, 5))
    face_locations = face_recognition.face_locations(frame)
    if len(face_locations) == 0:
        continue
    top, right, bottom, left = face_locations[0]
    frame_face = frame[top-padding:bottom+padding, left-padding:right+padding]
    image = cv2.cvtColor(frame_face, cv2.COLOR_BGR2RGB)

    img = pImage.fromarray(image, 'RGB')
    image_name = "john_cropped_faces_"+str(i)+'.png'

    image_path = "/content/faces/" + image_name
    img.save(image_path)
    faces_found = faces_found + 1
    faces_cropped_images.append(image_name)


In [43]:
if faces_found == 0:
    print("No Face Found")

In [17]:
SEQUENCE_LENGTH=20

In [12]:
import numpy as np

In [13]:
class validation_dataset(Dataset):
    def __init__(self,video_names,sequence_length=60,transform = None):
        self.video_names = video_names
        self.transform = transform
        self.count = sequence_length

    def __len__(self):
        return len(self.video_names)

    def __getitem__(self,idx):
        video_path = self.video_names[idx]
        frames = []
        a = int(100/self.count)
        first_frame = np.random.randint(0,a)
        for i,frame in enumerate(self.frame_extract(video_path)):
            #if(i % a == first_frame):
            faces = face_recognition.face_locations(frame)
            try:
              top,right,bottom,left = faces[0]
              frame = frame[top:bottom,left:right,:]
            except:
              pass
            frames.append(self.transform(frame))
            if(len(frames) == self.count):
                break
        """
        for i,frame in enumerate(self.frame_extract(video_path)):
            if(i % a == first_frame):
                frames.append(self.transform(frame))
        """
        # if(len(frames)<self.count):
        #   for i in range(self.count-len(frames)):
        #         frames.append(self.transform(frame))
        #print("no of frames", self.count)
        frames = torch.stack(frames)
        frames = frames[:self.count]
        return frames.unsqueeze(0)

    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path)
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image

In [14]:
im_size = 112
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
sm = nn.Softmax()

In [15]:
train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

In [18]:
video_dataset = validation_dataset(["/content/john.mp4"], sequence_length=SEQUENCE_LENGTH,transform= train_transforms)

In [19]:
inv_normalize =  transforms.Normalize(mean=-1*np.divide(mean,std),std=np.divide([1,1,1],std))


In [20]:
import matplotlib.pyplot as plt


In [21]:
def im_convert(tensor, video_file_name):
    """ Display a tensor as an image. """
    image = tensor.to("cpu").clone().detach()
    image = image.squeeze()
    image = inv_normalize(image)
    image = image.numpy()
    image = image.transpose(1,2,0)
    image = image.clip(0, 1)
    # This image is not used
    # cv2.imwrite(os.path.join(settings.PROJECT_DIR, 'uploaded_images', video_file_name+'_convert_2.png'),image*255)
    return image

In [22]:
def im_plot(tensor):
    image = tensor.cpu().numpy().transpose(1,2,0)
    b,g,r = cv2.split(image)
    image = cv2.merge((r,g,b))
    image = image*[0.22803, 0.22145, 0.216989] +  [0.43216, 0.394666, 0.37645]
    image = image*255.0
    plt.imshow(image.astype(int))
    plt.show()

In [23]:
def predict(model,img,path = './', video_file_name=""):
  fmap,logits = model(img.to('cuda'))
  img = im_convert(img[:,-1,:,:,:], video_file_name)
  params = list(model.parameters())
  weight_softmax = model.linear1.weight.detach().cpu().numpy()
  logits = sm(logits)
  _,prediction = torch.max(logits,1)
  confidence = logits[:,int(prediction.item())].item()*100
  print('confidence of prediction:',logits[:,int(prediction.item())].item()*100)
  return [int(prediction.item()),confidence]

In [32]:
import os
def plot_heat_map(i, model, img, path = './', video_file_name=''):
  fmap,logits = model(img.to('cuda'))
  params = list(model.parameters())
  weight_softmax = model.linear1.weight.detach().cpu().numpy()
  logits = sm(logits)
  _,prediction = torch.max(logits,1)
  idx = np.argmax(logits.detach().cpu().numpy())
  bz, nc, h, w = fmap.shape
  #out = np.dot(fmap[-1].detach().cpu().numpy().reshape((nc, h*w)).T,weight_softmax[idx,:].T)
  out = np.dot(fmap[i].detach().cpu().numpy().reshape((nc, h*w)).T,weight_softmax[idx,:].T)
  predict = out.reshape(h,w)
  predict = predict - np.min(predict)
  predict_img = predict / np.max(predict)
  predict_img = np.uint8(255*predict_img)
  out = cv2.resize(predict_img, (im_size,im_size))
  heatmap = cv2.applyColorMap(out, cv2.COLORMAP_JET)
  img = im_convert(img[:,-1,:,:,:], video_file_name)
  result = heatmap * 0.5 + img*0.8*255
  # Saving heatmap - Start
  heatmap_name = video_file_name+"_heatmap_"+str(i)+".png"
  image_name = os.path.join('/content/heatmap', heatmap_name)
  cv2.imwrite(image_name,result)
  # Saving heatmap - End
  result1 = heatmap * 0.5/255 + img*0.8
  r,g,b = cv2.split(result1)
  result1 = cv2.merge((r,g,b))
  return image_name

In [ ]:
heatmap_images = []
sequence_length=20
for i in range(0, len(["/content/john.mp4"])):
    output = ""
    print("<=== | Started Predicition | ===>")
    prediction = predict(model, video_dataset[i], './', "john")
    confidence = round(prediction[1], 1)
    print("<=== |  Predicition Done | ===>")
    print("<=== | Heat map creation started | ===>")
    for j in range(0, sequence_length):
        heatmap_images.append(plot_heat_map(j, model, video_dataset[i], './', "john"))
    if prediction[0] == 1:
        output = "REAL"
    else:
        output = "FAKE"
    print("Prediction : " , prediction[0],"==",output ,"Confidence : " , confidence)

<=== | Started Predicition | ===>


<ipython-input-23-c359ec5fe8ad>:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = sm(logits)


confidence of prediction: 99.68593120574951
<=== |  Predicition Done | ===>
<=== | Heat map creation started | ===>


In [31]:
heatmap_images

['/content/uploaded_images/john_heatmap_0.png',
 '/content/uploaded_images/john_heatmap_1.png',
 '/content/uploaded_images/john_heatmap_2.png',
 '/content/uploaded_images/john_heatmap_3.png',
 '/content/uploaded_images/john_heatmap_4.png',
 '/content/uploaded_images/john_heatmap_5.png',
 '/content/uploaded_images/john_heatmap_6.png',
 '/content/uploaded_images/john_heatmap_7.png',
 '/content/uploaded_images/john_heatmap_8.png',
 '/content/uploaded_images/john_heatmap_9.png',
 '/content/uploaded_images/john_heatmap_10.png',
 '/content/uploaded_images/john_heatmap_11.png',
 '/content/uploaded_images/john_heatmap_12.png',
 '/content/uploaded_images/john_heatmap_13.png',
 '/content/uploaded_images/john_heatmap_14.png',
 '/content/uploaded_images/john_heatmap_15.png',
 '/content/uploaded_images/john_heatmap_16.png',
 '/content/uploaded_images/john_heatmap_17.png',
 '/content/uploaded_images/john_heatmap_18.png',
 '/content/uploaded_images/john_heatmap_19.png']